# Thread join()

Wenn die Join-Methode aufgerufen wird, wird der aufrufende Thread blockiert, bis das Thread-Objekt, auf dem sie aufgerufen wurde
beendet wird. 

Wenn join() zum Beispiel von einem Haupt-Thread aufgerufen wird, wartet der Haupt-Thread 
solange, bis der Child-Thread, in dem join aufgerufen wurde, beendet wird. 
Die Bedeutung der join()-Methode liegt darin, dass der Haupt-Thread vor dem Child-Thread beendet werden kann, wenn join() nicht aufgerufen wird. 
kann der Haupt-Thread vor dem Child-Thread beendet werden, was zu unbestimmtem Verhalten von Programmen führt.

In [7]:
import threading
from dataclasses import dataclass
import time


@dataclass
class ConnectionThread(threading.Thread):    
    def __init__(self, address: str):
        self.address = address
        self.established = False
    
    def run(self):
        print("Ich bin der Child-Thread und beginne meine Arbeit")
        for _ in range(10):
            print(f"Verbinde zu {self.address}")
            time.sleep(1)
        self.established = True
        print("Ich bin der Child-Thread und meine Arbeit ist zuende")
        
if __name__=='__main__':
    connection = ConnectionThread("127.0.0.1:8000")
    my_thread = threading.Thread(target=connection.run)
    my_thread.start()
    
    print("Ich bin der Main-Thread und warte hier...")
    
    my_thread.join()
    
    print("Ich bin der Main-Thread und mache jetzt weiter ...")
    
    if connection.established:
        print("Verbindung Erfolgreich")
    else: print("Verbindung Fehlgeschlagen")
    

Ich bin der Child-Thread und beginne meine Arbeit
Verbinde zu 127.0.0.1:8000
Ich bin der Main-Thread und warte hier...
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Verbinde zu 127.0.0.1:8000
Ich bin der Child-Thread und meine Arbeit ist zuende
Ich bin der Main-Thread und mache jetzt weiter ...
Verbindung Erfolgreich


In [22]:
import random
from threading import Thread
import time

def create_connection(address: str) -> str:
    connection_established = random.choice([True, False])
    for _ in range(random.randint(1, 10)):
        time.sleep(1)
    if connection_established:
        return f"Verbindung zu {address} erfolgreich"
    else:
        return f"Verbindung zu {address} fehlgeschlagen"

class ConnectionThread(Thread):    
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}):
        Thread.__init__(self, group, target, name, args, kwargs)
    def run(self):
        if self._target != None:
            self._return = self._target(*self._args, **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return
        
if __name__=='__main__':
    thread_list: list[ConnectionThread] = []
    for i in range(10):
        thread_list.append(ConnectionThread(target=create_connection, args=(f"127.0.0.{i}:8000",)))
    
    for thread in thread_list:
        thread.start()

    print("Ich bin der Main-Thread und warte hier...")
    while any([thread.is_alive() for thread in thread_list]):
        time.sleep(1)
        # join all threads that are finished
        for thread in thread_list:
            if not thread.is_alive():
                print(thread.join())
                thread_list.remove(thread)

    print("Ich bin der Main-Thread und mache jetzt weiter ...")

Ich bin der Main-Thread und warte hier...
Verbindung zu 127.0.0.3:8000 fehlgeschlagen
Verbindung zu 127.0.0.2:8000 erfolgreich
Verbindung zu 127.0.0.1:8000 fehlgeschlagen
Verbindung zu 127.0.0.4:8000 fehlgeschlagen
Verbindung zu 127.0.0.6:8000 fehlgeschlagen
Verbindung zu 127.0.0.8:8000 fehlgeschlagen
Verbindung zu 127.0.0.9:8000 erfolgreich
Verbindung zu 127.0.0.5:8000 fehlgeschlagen
Verbindung zu 127.0.0.0:8000 fehlgeschlagen
Ich bin der Main-Thread und mache jetzt weiter ...
